In [2]:
class Constraint(object):
    """A Constraint consists of
    * scope: a tuple of variables
    * condition: a function that can applied to a tuple of values
    for the variables
    """
    def __init__(self, scope, condition):
        self.scope = scope
        self.condition = condition

    def __repr__(self):
        return self.condition.__name__ + str(self.scope)

    def holds(self, assignment):
        """returns the value of Constraint con evaluated in assignment.

        precondition: all variables are assigned in assignment
        """
        return self.condition(*tuple(assignment[v] for v in self.scope))


In [3]:
class CSP(object):
    """A CSP consists of
    * domains, a dictionary that maps each variable to its domain
    * constraints, a list of constraints
    * variables, a set of variables
    * var_to_const, a variable to set of constraints dictionary
    """
    def __init__(self,domains,constraints):
        """domains is a variable:domain dictionary
        constraints is a list of constriants
        """
        self.variables = set(domains)
        self.domains = domains
        self.constraints = constraints
        self.var_to_const = {var:set() for var in self.variables}
        for con in constraints:
            for var in con.scope:
                self.var_to_const[var].add(con)

    def __str__(self):
        """string representation of CSP"""
        return str(self.domains)

    def __repr__(self):
        """more detailed string representation of CSP"""
        return "CSP({}, {})".format(self.domains, [str(c) for c in self.constraints])

    def consistent(self, assignment):
        """assignment is a variable:value dictionary
        returns True if all of the constraints that can be evaluated
                        evaluate to True given assignment.
        """
        return all(con.holds(assignment)
                    for con in self.constraints
                    if all(v in  assignment  for v in con.scope))


In [4]:
from operator import lt, ne, eq, gt

def ne_(val):
    """not equal value"""
    # nev = lambda x: x != val   # alternative definition
    # nev = partial(neq,val)     # another alternative definition
    def nev(x):
        return val != x
    nev.__name__ = str(val)+"!="      # name of the function 
    return nev

def is_(val):
    """is a value"""
    # isv = lambda x: x == val   # alternative definition
    # isv = partial(eq,val)      # another alternative definition
    def isv(x):
        return val == x
    isv.__name__ = str(val)+"=="
    return isv

csp0 = CSP(
    {
        'X': {1, 2, 3}, 'Y': {1, 2, 3}, 'Z': {1, 2, 3}
    },
    [
        Constraint(('X', 'Y'), lt),
        Constraint(('Y', 'Z'), lt)
    ]
)

C0 = Constraint(('A', 'B'), lt)
C1 = Constraint(('B',), ne_(2))
C2 = Constraint(('B', 'C'), lt)
csp1 = CSP(
    {'A': {1, 2, 3, 4}, 'B': {1, 2, 3, 4}, 'C': {1, 2, 3, 4}},
    [C0, C1, C2]
)

csp2 = CSP(
    {'A': {1, 2, 3, 4}, 'B': {1, 2, 3, 4}, 'C': {1, 2, 3, 4}, 'D': {1, 2, 3, 4}, 'E': {1, 2, 3, 4}},
    [
        Constraint(('B',), ne_(3)),
        Constraint(('C',), ne_(2)),
        Constraint(('A', 'B'), ne),
        Constraint(('B', 'C'), ne),
        Constraint(('C', 'D'), lt),
        Constraint(('A', 'D'), eq),
        Constraint(('A', 'E'), gt),
        Constraint(('B', 'E'), gt),
        Constraint(('C', 'E'), gt),
        Constraint(('D', 'E'), gt),
        Constraint(('B', 'D'), ne)
    ]
)

csp3 = CSP(
    {'A': {1, 2, 3, 4}, 'B': {1, 2, 3, 4}, 'C': {1, 2, 3, 4}, 'D': {1, 2, 3, 4}, 'E': {1, 2, 3, 4}},
    [
        Constraint(('A','B'), ne),
        Constraint(('A','D'), lt),
        Constraint(('A','E'), lambda a, e: (a - e) % 2 == 1), # A-E is odd
        Constraint(('B','E'), lt),
        Constraint(('D','C'), lt),
        Constraint(('C','E'), ne),
        Constraint(('D','E'), ne)
    ]
)

def adjacent(x, y):
   """True when x and y are adjacent numbers"""
   return abs(x - y) == 1

csp4 = CSP(
    {'A': {1, 2, 3, 4, 5}, 'B': {1, 2, 3, 4, 5}, 'C': {1, 2, 3, 4, 5}, 'D': {1, 2, 3, 4, 5}, 'E': {1, 2, 3, 4, 5}},
    [
        Constraint(('A','B'), adjacent),
        Constraint(('B','C'), adjacent),
        Constraint(('C','D'), adjacent),
        Constraint(('D','E'), adjacent),
        Constraint(('A','C'), ne),
        Constraint(('B','D'), ne),
        Constraint(('C','E'), ne)
    ]
)

def meet_at(p1, p2):
    """returns a function that is true when the words meet at the postions p1, p2
    """
    def meets(w1, w2):
        return w1[p1] == w2[p2]
    meets.__name__ = "meet_at({}, {})".format(p1, p2)
    return meets

crossword1 = CSP(
    {
        'one_across': {'ant', 'big', 'bus', 'car', 'has'},
        'one_down': {'book', 'buys', 'hold', 'lane', 'year'},
        'two_down': {'ginger', 'search', 'symbol', 'syntax'},
        'three_across': {'book', 'buys', 'hold', 'land', 'year'},
        'four_across': {'ant', 'big', 'bus', 'car', 'has'}
    },
    [
        Constraint(('one_across', 'one_down'), meet_at(0, 0)),
        Constraint(('one_across', 'two_down'), meet_at(2, 0)),
        Constraint(('three_across', 'two_down'), meet_at(2, 2)),
        Constraint(('three_across', 'one_down'), meet_at(0, 2)),
        Constraint(('four_across', 'two_down'), meet_at(0, 4))
    ]
)

words = {
    'ant', 'big', 'bus', 'car', 'has','book', 'buys', 'hold',
    'lane', 'year', 'ginger', 'search', 'symbol', 'syntax'
}
           
def is_word(*letters, words=words):
    """is true if the letters concatenated form a word in words"""
    return "".join(letters) in words

letters = [
    "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m",
    "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"
]

crossword1d = CSP(
    {
        'p00': letters, 'p10': letters, 'p20': letters,                 # first row
        'p01': letters, 'p21': letters,                                 # second row
        'p02': letters, 'p12': letters, 'p22': letters, 'p32': letters, # third row
        'p03': letters, 'p23': letters,                                 # fourth row
        'p24': letters, 'p34': letters, 'p44': letters,                 # fifth row
        'p25': letters                                                  # sixth row
    },
    [
        Constraint(('p00', 'p10', 'p20'), is_word),                      # 1-across
        Constraint(('p00', 'p01', 'p02', 'p03'), is_word),               # 1-down
        Constraint(('p02', 'p12', 'p22', 'p32'), is_word),               # 3-across
        Constraint(('p20', 'p21', 'p22', 'p23', 'p24', 'p25'), is_word), # 2-down
        Constraint(('p24', 'p34', 'p44'), is_word)                       # 4-across
    ]
)
               
def test(CSP_solver, csp=csp1, solutions=[{'A': 1, 'B': 3, 'C': 4}, {'A': 2, 'B': 3, 'C': 4}]):
    """CSP_solver is a solver that takes a csp and returns a solution
    csp is a constraint satisfaction problem
    solutions is the list of all solutions to csp
    This tests whether the solution returned by CSP_solver is a solution.
    """
    print("Testing csp with", CSP_solver.__doc__)
    sol0 = CSP_solver(csp)
    print("Solution found:", sol0)
    assert sol0 in solutions, "Solution not correct for {}".format(csp)
    print("Passed unit test")


In [5]:
from lib.sp import Arc, Search_problem
from lib.utilities import dict_union

class Search_from_CSP(Search_problem):
    """A search problem directly from the CSP.

    A node is a variable:value dictionary"""
    def __init__(self, csp, variable_order=None):
        self.csp=csp
        if variable_order:
            assert set(variable_order) == set(csp.variables)
            assert len(variable_order) == len(csp.variables)
            self.variables = variable_order
        else:
            self.variables = list(csp.variables)

    def is_goal(self, node):
        """returns whether the current node is a goal for the search
        """
        return len(node) == len(self.csp.variables)
    
    def start_node(self):
        """returns the start node for the search
        """
        return {}
    
    def neighbors(self, node):
        """returns a list of the neighboring nodes of node.
        """
        var = self.variables[len(node)] # the next variable
        res = []
        for val in self.csp.domains[var]:
            new_env = dict_union(node, {var: val})  #dictionary union
            if self.csp.consistent(new_env):
                res.append(Arc(node, new_env))
        return res


In [6]:
class Path(object):
    """A path is either a node or a path followed by an arc"""
    
    def __init__(self, initial, arc=None):
        """initial is either a node (in which case arc is None) or
        a path (in which case arc is an object of type Arc)"""
        self.initial = initial
        self.arc = arc
        if arc is None:
            self.cost = 0
        else:
            self.cost = initial.cost + arc.cost

    def end(self):
        """returns the node at the end of the path"""
        if self.arc is None:
            return self.initial
        else:
            return self.arc.to_node

    def nodes(self):
        """enumerates the nodes for the path.
        This starts at the end and enumerates nodes in the path backwards."""
        current = self
        while current.arc is not None:
            yield current.arc.to_node
            current = current.initial
        yield current.initial

    def initial_nodes(self):
        """enumerates the nodes for the path before the end node.
        This starts at the end and enumerates nodes in the path backwards."""
        if self.arc is not None:
            for nd in self.initial.nodes(): yield nd     # could be "yield from"
        
    def __repr__(self):
        """returns a string representation of a path"""
        if self.arc is None:
            return str(self.initial)
        elif self.arc.action:
            return '{}\n   --{}--> {}'.format(self.initial, self.arc.action, self.arc.to_node)
        else:
            return '{} --> {}'.format(self.initial, self.arc.to_node)


In [7]:
from lib.display import Displayable, visualize

class Searcher(Displayable):
    """returns a searcher for a problem.
    Paths can be found by repeatedly calling search().
    This does depth-first search unless overridden
    """
    
    def __init__(self, problem):
        """creates a searcher from a problem
        """
        self.problem = problem
        self.initialize_frontier()
        self.num_expanded = 0
        self.add_to_frontier(Path(problem.start_node()))
        super().__init__()

    def initialize_frontier(self):
        self.frontier = []
        
    def empty_frontier(self):
        return self.frontier == []
        
    def add_to_frontier(self, path):
        self.frontier.append(path)
        
    @visualize
    def search(self):
        """returns (next) path from the problem's start node
        to a goal node. 
        Returns None if no path exists.
        """
        while not self.empty_frontier():
            path = self.frontier.pop()
            self.display(2, "Expanding:",path,"(cost:",path.cost,")")
            self.num_expanded += 1
            if self.problem.is_goal(path.end()):    # solution found
                self.display(1, self.num_expanded, "paths have been expanded and",
                            len(self.frontier), "paths remain in the frontier")
                self.solution = path   # store the solution found
                return path
            else:
                neighs = self.problem.neighbors(path.end())
                self.display(3,"Neighbors are", neighs)
                for arc in reversed(list(neighs)):
                    self.add_to_frontier(Path(path, arc))
                self.display(3,"Frontier:",self.frontier)
        self.display(1, "No (more) solutions. Total of", self.num_expanded, "paths expanded.")


In [13]:
class_csp = CSP(
    {'A': {1, 2, 3, 4}, 'B': {1, 2, 3, 4}, 'C': {1, 2, 3, 4}},
    [
        Constraint(('A', 'B'), lt),
        Constraint(('B', 'C'), lt)
    ]
)


def dfs_solver(csp):
    """depth-first search solver"""
    path = Searcher(Search_from_CSP(csp)).search()
    if path is not None:
        return path.end()
    else:
        return None

if __name__ == "__main__":
    test(dfs_solver, class_csp, [
        {'A': 1, 'B': 2, 'C': 4},
        {'A': 1, 'B': 2, 'C': 3}
    ])

    ## Test Solving CSPs with Search:
    # searcher1 = Searcher(Search_from_CSP(csp1))
    # print(searcher1.search())  # get next solution

    # searcher2 = Searcher(Search_from_CSP(csp2))
    # print(searcher2.search())  # get next solution

    # searcher3 = Searcher(Search_from_CSP(crossword1))
    # print(searcher3.search())  # get next solution

    # searcher4 = Searcher(Search_from_CSP(crossword1d))
    # print(searcher4.search())  # get next solution (warning: slow)


Testing csp with depth-first search solver
Expanding: {} (cost: 0 )
Neighbors are [{} --> {'C': 1}, {} --> {'C': 2}, {} --> {'C': 3}, {} --> {'C': 4}]
Frontier: [{} --> {'C': 4}, {} --> {'C': 3}, {} --> {'C': 2}, {} --> {'C': 1}]
Expanding: {} --> {'C': 1} (cost: 1 )
Neighbors are []
Frontier: [{} --> {'C': 4}, {} --> {'C': 3}, {} --> {'C': 2}]
Expanding: {} --> {'C': 2} (cost: 1 )
Neighbors are [{'C': 2} --> {'C': 2, 'B': 1}]
Frontier: [{} --> {'C': 4}, {} --> {'C': 3}, {} --> {'C': 2} --> {'C': 2, 'B': 1}]
Expanding: {} --> {'C': 2} --> {'C': 2, 'B': 1} (cost: 2 )
Neighbors are []
Frontier: [{} --> {'C': 4}, {} --> {'C': 3}]
Expanding: {} --> {'C': 3} (cost: 1 )
Neighbors are [{'C': 3} --> {'C': 3, 'B': 1}, {'C': 3} --> {'C': 3, 'B': 2}]
Frontier: [{} --> {'C': 4}, {} --> {'C': 3} --> {'C': 3, 'B': 2}, {} --> {'C': 3} --> {'C': 3, 'B': 1}]
Expanding: {} --> {'C': 3} --> {'C': 3, 'B': 1} (cost: 2 )
Neighbors are []
Frontier: [{} --> {'C': 4}, {} --> {'C': 3} --> {'C': 3, 'B': 2}]
Expa